# Retirada do efeito de fontes regionais

Notebook que demonstrará a retirada do efeito de fontes regionais profundas

# Importando bibliotecas

Importando as bibliotecas que serão utilizadas para manipulação e visualização dos dados de gravidade.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import harmonica as hm
import cartopy.crs as ccrs
import matplotlib.colors as colors
import boule as bl
import pyproj
import verde as vd

# Carregando os dados

In [ ]:
fname = 'dataset/gravity_earth_brazilian_coast.gdf'
gravity = hm.load_icgem_gdf(fname)

In [ ]:
fname = 'dataset/geoid_brazilian_coast.gdf'
geoid = hm.load_icgem_gdf(fname)

In [ ]:
fname = 'dataset/topography_brazilian_coast.gdf'
topo = hm.load_icgem_gdf(fname)

### Armazenando os dados em um array e manipulando os dados

In [ ]:
lat = gravity.latitude
lon = gravity.longitude
elevation = gravity.h_over_geoid
data = gravity.gravity_earth
geoidal_height = geoid.geoid
topography = topo.topography_shm

In [ ]:
longitude, latitude = np.meshgrid(lon.values,lat.values)

In [ ]:
field = data.values.reshape(longitude.shape)

# Gravidade real (Costa Brasileira)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Gravity (Brazilian Coast)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([309.38, 329.03, -29.231, -13.831], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,field,
                   100,transform=ccrs.PlateCarree(),
                   cmap='viridis')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/gravity_earth_brazil_coast.png',dpi=200,bbox_inches='tight')

plt.show()

# Gravidade Normal sobre a superfície terrestre

In [ ]:
normal_gravity = bl.WGS84.normal_gravity(latitude, elevation)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Normal gravity (Brazilian Coast)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([309.38, 329.03, -29.231, -13.831], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,normal_gravity,
                   100,transform=ccrs.PlateCarree(),
                   cmap='viridis')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/normal_gravity_brazil_coast.png',dpi=200,bbox_inches='tight')
plt.show()

# Distúrbio de gravidade 

In [ ]:
disturbance = field - normal_gravity

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Gravity Disturbance (Brazilian Coast)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([309.38, 329.03, -29.231, -13.831], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(lon,lat,disturbance,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='RdBu_r')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/disturbance_brazil_coast.png',dpi=200,bbox_inches='tight')

plt.show()

# Topografia

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Topography (Brazilian Coast)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([309.38, 329.03, -29.231, -13.831], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,topography,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='terrain')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('meters',fontsize=10)

plt.savefig('images/topography_brazil_coast.png',dpi=200,bbox_inches='tight')

plt.show()

# Anomalia Bouguer

In [ ]:
reference = geoidal_height + topography
bouguer_correction = hm.bouguer_correction(reference)
bouguer_anomaly = disturbance - bouguer_correction

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Bouguer Anomaly (Brazilian Coast)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([309.38, 329.03, -29.231, -13.831], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,bouguer_anomaly,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='RdBu_r')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/bouguer_brazil_coast.png',dpi=200,bbox_inches='tight')

plt.show()

# Retirando o efeito de fontes regionais profundas

In [ ]:
heq = topography.values.copy()
heq[heq < 0] *= (2670 - 1040)/2670

In [ ]:
A = np.ones((heq.size, 2))
A[:, 0] = heq.ravel()
p = np.linalg.solve(A.T.dot(A), A.T.dot(bouguer_anomaly.values.ravel()))
efeito_moho = A.dot(p)

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))

ax.plot(heq.ravel(),bouguer_anomaly.values.ravel(),'.k')
ax.plot(heq.ravel(),efeito_moho.ravel(),'-')
ax.set_xlabel('Topografia equivalente (m)')
ax.set_ylabel('Anomalia Bouguer (mGal)')

plt.show()

In [ ]:
residuals = bouguer_anomaly - efeito_moho.reshape(longitude.shape)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Bouguer Anomaly (w/o Moho effect)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([309.38, 329.03, -29.231, -13.831], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,residuals,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='seismic')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/bouguer_without_moho.png',dpi=200,bbox_inches='tight')

plt.show()